Paint con manos

In [5]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import math
import time


In [37]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
  
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Paleta de colores
paleta = cv2.imread("media/fotos/paleta.png") 
paleta = cv2.resize(paleta, (250, 70))

#Añadir delay al cambiar color
last_d_key_press_time = time.time()

# Diccionario con los colores, su codigo bgr y su posicion en el frame
colors_dict = {
    'red': ((0, 0, 248), (204, 15)),
    'orange': ((0, 102, 255), (204, 48)),
    'salmon': ((104, 0, 255), (235, 15)),
    'pink': ((210, 0, 254), (235, 48)),
    'purple': ((176, 0, 161), (265, 15)),
    'deep purple': ((165, 40, 102), (265, 48)),
    'deep blue': ((206, 52, 53), (295, 15)),
    'blue': ((216, 172, 1), (295, 48)),
    'green': ((0, 172, 0), (326, 15)),
    'soft blue': ((137, 161, 1), (326, 48)),
    'yellow': ((0, 205, 255), (356, 15)),
    'deep green': ((0, 213, 150), (356, 48)),
    'almost black': ((34, 5, 36), (387, 15)),
    'soft pink': ((243, 192, 247), (387, 48)),
    'black': ((0, 0, 0), (417, 15)),
    'white': ((255, 255, 255), (417, 48))
}

color_names = list(colors_dict.keys())
current_color_index = 0

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    #max_num_hands=2,
    min_tracking_confidence=0.5) as hands:
  

  #aqui guardaremos los circulos que vamos dibujando  
  list_of_draws = []

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue


                     
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    image[10:10 +70, 200:200 +250] = paleta
    color_name = color_names[current_color_index]

    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        #print(hand_landmarks)
        
        
        thumb_x = hand_landmarks.landmark[4].x 
        thumb_y = hand_landmarks.landmark[4].y
        thumb_z = hand_landmarks.landmark[4].z 

        index_x = hand_landmarks.landmark[8].x
        index_y = hand_landmarks.landmark[8].y 
        index_z = hand_landmarks.landmark[8].z

        #print("pulgar:/nx: ", thumb_x, "/ny: ", thumb_y, "/nz: ", thumb_z)
         
    
        
        # Calcular la distancia entre la punta del pulgar y la del índice
      distance = math.sqrt((thumb_x - index_x)**2 + (thumb_y - index_y)**2)# + (thumb_z - index_z)**2)   
      #print(distance)

      # Establecer un umbral para considerar que están juntos
      threshold = 0.07

      if distance < threshold:
          midpoint = ((thumb_x + index_x) / 2, (thumb_y + index_y) / 2, (thumb_z + index_z) / 2, colors_dict[color_name][0])
          list_of_draws.append(midpoint)

      elif list_of_draws and list_of_draws[-1] != (-1, -1, -1): 
        list_of_draws.append((-1, -1, -1, (255, 255, 255)))


    # Juntar todos los puntos que se han creado de forma consecutivo para crear el dibujo
    x, y, z = (-1, -1, -1)
    for position in list_of_draws:
      prev_x, prev_y, prev_z = x, y, z
      x, y, z, color_point = position

      # No unir puntos si se paro de dibujar
      if (x, y, z) == (-1, -1, -1) or (prev_x == -1 and prev_y == -1 and prev_z == -1):
        continue


      
      height, width, deep = image.shape
      midpoint_x, midpoint_y, midpoint_z= int(x * width), int(y * height), int(z * deep)
      prev_midpoint_x, prev_midpoint_y, prev_midpoint_z= int(prev_x * width), int(prev_y * height), int(prev_z * deep)
      
      # Evitar dibujar sobre la paleta
      if 190 < prev_midpoint_x < 460 and 190 < midpoint_x < 460 and 0 < prev_midpoint_y < 90 and 0 < midpoint_y < 90:
        continue 
      
      cv2.line(image, (midpoint_x, midpoint_y), (prev_midpoint_x, prev_midpoint_y), color_point, 5)
      
    x_color_square, y_color_square = colors_dict[color_name][1]  
    cv2.rectangle(image, (x_color_square, y_color_square), (x_color_square+27, y_color_square+27), (162, 0, 0), 2)
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    
    # Si se pulsa 'z' deshacer lo que se ha dibujado    
    if (cv2.waitKey(1) & 0xFF  == ord('z')):
      while list_of_draws and list_of_draws[-1][0] == (-1, -1, -1):
        list_of_draws.pop()
      
      if list_of_draws:
        list_of_draws.pop() 

    elif (cv2.waitKey(1) & 0xFF  == ord('d')):
      current_time = time.time()
      if current_time - last_d_key_press_time >= 1.1:
          
          print("Cambia color")
          current_color_index = (current_color_index +1) % len(color_names)
          
          last_d_key_press_time = current_time

    # Terminar si se pulsa 'q'
    elif cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()


Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color
Cambia color


In [12]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
  
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Paleta de colores
paleta = cv2.imread("media/fotos/paleta.png") 
paleta = cv2.resize(paleta, (250, 70))

colors_dict = {
    'red': (248, 0, 0),
    'orange': (255, 102, 0),
    'salmon': (255, 0, 104),
    'pink': (254, 0, 210),
    'purple': (161, 0, 176),
    'deep purple': (102, 40, 165),
    'blue': (1, 172, 216),
    'deep blue': (53, 52, 206),
    'green': (0, 172, 0),
    'soft blue': (1, 161, 137),
    'deep green': (150, 213, 0),
    'yellow': (255, 205, 0),
    'soft pink': (247, 192, 243),
    'almost black': (36, 5, 34),
    'black': (0, 0, 0),
    'white': (255, 255, 255)
}

current_color = (0, 255, 0)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    #max_num_hands=2,
    min_tracking_confidence=0.5) as hands:
  

  #aqui guardaremos los circulos que vamos dibujando  
  list_of_draws = []

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue


                     
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    image[10:10 +70, 200:200 +250] = paleta

    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        #print(hand_landmarks)
        
        
        thumb_x = hand_landmarks.landmark[4].x 
        thumb_y = hand_landmarks.landmark[4].y
        thumb_z = hand_landmarks.landmark[4].z 

        index_x = hand_landmarks.landmark[8].x
        index_y = hand_landmarks.landmark[8].y 
        index_z = hand_landmarks.landmark[8].z

        #print("pulgar:/nx: ", thumb_x, "/ny: ", thumb_y, "/nz: ", thumb_z)
         
    
        
        # Calcular la distancia entre la punta del pulgar y la del índice
      distance = math.sqrt((thumb_x - index_x)**2 + (thumb_y - index_y)**2 + (thumb_z - index_z)**2)   
      #print(distance)
      
      
      print(index_z)
# Calcular la distancia tridimensional teniendo en cuenta la profundidad
      distance_3d = distance * index_z ** 2
      print(distance_3d)
          # Establecer un umbral para considerar que están juntos
      threshold = 0.07

      if distance_3d < threshold:# and index_z < deep_threshold:
          unido = True
          midpoint = ((thumb_x + index_x) / 2, (thumb_y + index_y) / 2, (thumb_z + index_z) / 2)
          list_of_draws.append(midpoint)

      else: 
        if unido == True:
          unido = False
          list_of_draws.append((-1, -1, -1))
    else: 
      if unido == True:
        unido = False
        list_of_draws.append((-1, -1, -1))


    # Juntar todos los puntos que se han creado de forma consecutivo para crear el dibujo
    x, y, z = (-1, -1, -1)
    for position in list_of_draws:
      prev_x, prev_y, prev_z = x, y, z
      x, y, z = position
      if position == (-1, -1, -1) or (prev_x == -1 and prev_y == -1 and prev_z == -1):
        continue
      
      height, width, deep = image.shape
      midpoint_x, midpoint_y, midpoint_z= int(x * width), int(y * height), int(z * deep)
      prev_midpoint_x, prev_midpoint_y, prev_midpoint_z= int(prev_x * width), int(prev_y * height), int(prev_z * deep)
      
      cv2.line(image, (midpoint_x, midpoint_y), (prev_midpoint_x, prev_midpoint_y), (0, 255, 0), 5)
      
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))

    key = cv2.waitKey(1) & 0xFF

    if key == 8 and list_of_draws:  # Tecla de retroceso
        print("Funciona retroceso")
        while list_of_draws and list_of_draws[-1] == (-1, -1, -1):
            list_of_draws.pop()
        list_of_draws.pop()
      
      if list_of_draws:
        list_of_draws.pop() 

    elif key == ord('z') and list_of_draws:  # Tecla 'z'
        print("Funciona 'z'")
        if list_of_draws[-1] == (-1, -1, -1):
            list_of_draws.pop()
        while list_of_draws and list_of_draws[-1] != (-1, -1, -1):
            list_of_draws.pop()

    elif key == ord('q'):  # Tecla 'q'
        break
cap.release()
cv2.destroyAllWindows()

NameError: name 'unido' is not defined